In [24]:
import yfinance as yf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Veriyi yükleme
# Veri çekme
stock = "THYAO.IS"
start_date = "2012-02-02"
end_date = "2023-12-02"
bist30_data = yf.download(stock, start=start_date, end=end_date)
df = bist30_data['Close'].reset_index()
prices = bist30_data["Close"].values.reshape(-1, 1)
# Veriyi işleme
# ...

# Normalizasyon
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

# Veriyi eğitim ve test setlerine ayırma
train_size = int(len(scaled_data) * 0.80)
train_data, test_data = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :]

# Veriyi zaman serileri haline getirme
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Veriyi LSTM modeline uygun hale getirme
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Model oluşturma
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=1, batch_size=1)

# Test seti üzerinde tahminleme
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

valid = df[len(df) - len(predictions):].copy()
valid['Predictions'] = np.nan  # Tahminlerle aynı uzunlukta bir sütun ekleyin, ancak başlangıçta NaN değerleriyle doldurun
valid['Predictions'] = predictions

# Görselleştirme
train = df[:train_size]
valid = df[train_size:]
valid['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

# Hata miktarını hesaplama
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print(f'Root Mean Squared Error (RMSE): {rmse}')


[*********************100%%**********************]  1 of 1 completed
16/16 [==============================] - 1s 18ms/step


ValueError: Length of values (509) does not match length of index (609)

In [5]:
predictions

array([[ 17.827152],
       [ 17.892273],
       [ 18.034582],
       [ 18.100386],
       [ 18.073856],
       [ 17.969383],
       [ 17.895372],
       [ 17.876692],
       [ 17.927418],
       [ 18.042166],
       [ 18.228527],
       [ 18.534819],
       [ 19.031397],
       [ 19.677635],
       [ 20.405235],
       [ 21.118818],
       [ 21.799961],
       [ 22.510326],
       [ 23.056404],
       [ 23.345875],
       [ 23.25056 ],
       [ 22.76178 ],
       [ 22.027424],
       [ 21.328249],
       [ 20.773623],
       [ 20.380455],
       [ 20.177681],
       [ 20.081495],
       [ 20.03703 ],
       [ 20.134193],
       [ 20.425676],
       [ 20.933971],
       [ 21.641851],
       [ 22.453068],
       [ 23.368818],
       [ 24.316135],
       [ 25.198412],
       [ 26.061724],
       [ 26.832352],
       [ 27.459818],
       [ 27.830019],
       [ 27.975853],
       [ 27.997765],
       [ 27.918854],
       [ 27.680878],
       [ 27.324692],
       [ 26.986612],
       [ 26.8

In [20]:
valid['Predictions']

KeyError: 'Predictions'

In [21]:
valid

,Date,Close
2436,2021-06-28,13.86
2437,2021-06-29,13.39
2438,2021-06-30,13.37
2439,2021-07-01,13.75
2440,2021-07-02,13.74
...,...,...
3040,2023-11-27,260.50
3041,2023-11-28,261.00
3042,2023-11-29,255.00
3043,2023-11-30,251.00


In [22]:
(predictions)

array([[ 16.999308],
       [ 17.085793],
       [ 17.22119 ],
       [ 17.26862 ],
       [ 17.231627],
       [ 17.131966],
       [ 17.070065],
       [ 17.058004],
       [ 17.1025  ],
       [ 17.196177],
       [ 17.347324],
       [ 17.601988],
       [ 18.023014],
       [ 18.56834 ],
       [ 19.180096],
       [ 19.781017],
       [ 20.367546],
       [ 21.002052],
       [ 21.500452],
       [ 21.790144],
       [ 21.761232],
       [ 21.413462],
       [ 20.878492],
       [ 20.392155],
       [ 20.016903],
       [ 19.744898],
       [ 19.597553],
       [ 19.499994],
       [ 19.417177],
       [ 19.448318],
       [ 19.643908],
       [ 20.024391],
       [ 20.575888],
       [ 21.21607 ],
       [ 21.960255],
       [ 22.747711],
       [ 23.498053],
       [ 24.2637  ],
       [ 24.9726  ],
       [ 25.578989],
       [ 25.977476],
       [ 26.20285 ],
       [ 26.340277],
       [ 26.393667],
       [ 26.296965],
       [ 26.086512],
       [ 25.882944],
       [ 25.8

In [18]:
predictions.isna().sum()

AttributeError: 'numpy.ndarray' object has no attribute 'isna'